# Slide-Seq Mouse Olfactory Bulb - multiple pucks

This example uses TACCO to annotate and analyse mouse olfactory bulb Slide-Seq data [Wang et al.] with mouse olfactory bulb scRNA-seq data [Tepe et al.] as reference.

[Wang et al.]: Wang IH, Murray E, Andrews G, Jiang HC et al. Spatial transcriptomic reconstruction of the mouse olfactory glomerular map suggests principles of odor processing. Nat Neurosci 2022 Apr;25(4):484-492. PMID: 35314823

[Tepe et al.]: Tepe B, Hill MC, Pekarek BT, Hunt PJ et al. Single-Cell RNA-Seq of Mouse Olfactory Bulb Reveals Cellular Heterogeneity and Activity-Dependent Molecular Census of Adult-Born Neurons. Cell Rep 2018 Dec 4;25(10):2689-2703.e3. PMID: 30517858

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore','invalid value encountered in true_divide')

import pandas as pd
import numpy as np
import anndata as ad

import tacco as tc

## Load data

In [ ]:
# The notebook expects to be executed either in the workflow directory or in the repository root folder
data_path = 'results/slideseq_mouse_olfactory_bulb'
if not os.path.exists(data_path):
    data_path = f'../../{data_path}'

if not os.path.exists(data_path):
    raise ValueError(f'The path to the data for the slideseq_mouse_olfactory_bulb cannot be found!')


In [ ]:
reference = ad.read(f'{data_path}/reference.h5ad')

In [ ]:
pucks = ad.concat({f.split('.')[0]: ad.read(f'{data_path}/{f}') for f in os.listdir(data_path) if f.startswith('puck_')},label='puck',index_unique='-')

In [ ]:
pucks.obs['replicate'] = pucks.obs['puck'].str.split('_').str[1].astype(int)
pucks.obs['slide'] = pucks.obs['puck'].str.split('_').str[2].astype(int)
unique_replicates = sorted(pucks.obs['replicate'].unique())
unique_slides = sorted(pucks.obs['slide'].unique())
pucks.obs['replicate'] = pucks.obs['replicate'].astype(pd.CategoricalDtype(unique_replicates,ordered=True))
pucks.obs['slide'] = pucks.obs['slide'].astype(pd.CategoricalDtype(unique_slides,ordered=True))
pucks.obs['puck'] = pucks.obs['puck'].astype(pd.CategoricalDtype([f'puck_{r}_{s}' for r in reversed(unique_replicates) for s in unique_slides],ordered=True))

## Get a first impression of the spatial data

Plot total counts

In [ ]:
pucks.obs['total_counts'] = tc.sum(pucks.X,axis=1)
pucks.obs['log10_counts'] = np.log10(1+pucks.obs['total_counts'])

In [ ]:
fig,axs=tc.pl.subplots(len(unique_slides),len(unique_replicates), x_padding=1.5)
fig = tc.pl.scatter(pucks, 'log10_counts', group_key='puck', cmap='viridis', cmap_vmin_vmax=[1,3], ax=axs[::-1].reshape((1,40)));
fig.savefig(f'{data_path}/scatter_counts.pdf',bbox_inches='tight')

Plot marker genes

In [ ]:
cluster2type = reference.obs[['ClusterName','type']].drop_duplicates().groupby('type')['ClusterName'].agg(lambda x: list(x.to_numpy()))
type2long = reference.obs[['type','long']].drop_duplicates().groupby('long')['type'].agg(lambda x: list(x.to_numpy()))

In [ ]:
marker_map = {}
for k,v in type2long.items():
    genes = []
    if '(' in k:
        genes = k.split('(')[-1].split(')')[0].split('/')
    marker_map[v[0]] = [g[:-1] for g in genes]

In [ ]:
pucks.obsm['type_mrk'] = pd.DataFrame(0.0, index=pucks.obs.index, columns=sorted(reference.obs['type'].unique()))
for k,v in marker_map.items():
    for g in v:
        pucks.obsm['type_mrk'][k] += pucks[:,g].X.A.flatten()
    total = pucks.obsm['type_mrk'][k].sum()
    if total > 0:
        pucks.obsm['type_mrk'][k] /= total

In [ ]:
fig,axs=tc.pl.subplots(len(unique_slides),len(unique_replicates))
tc.pl.scatter(pucks, 'type_mrk', group_key='puck', ax=axs[::-1].reshape((1,40)), compositional=True);
fig.savefig(f'{data_path}/scatter_marker.pdf',bbox_inches='tight')

## Annotate the spatial data with compositions of cell types

Annotation is done on cluster level to capture variation within a cell type...

In [ ]:
tc.tl.annotate(pucks,reference,'ClusterName',result_key='ClusterName',)

... and then aggregated to cell type level for visualization

In [ ]:
tc.utils.merge_annotation(pucks, 'ClusterName', cluster2type, 'type');
tc.utils.merge_annotation(pucks, 'type', type2long, 'long');

In [ ]:
fig,axs=tc.pl.subplots(len(unique_slides),len(unique_replicates))
tc.pl.scatter(pucks, 'type', group_key='puck', ax=axs[::-1].reshape((1,40)));
fig.savefig(f'{data_path}/scatter_type.pdf',bbox_inches='tight')

## Analyse co-occurrence and neighbourhips

Calculate distance matrices per sample and evaluate different spatial metrics on that. Using sparse distance matrices is useful if one is interested only in small distances relative to the sample size.

In [ ]:
tc.tl.co_occurrence(pucks, 'type', sample_key='puck', result_key='type-type',delta_distance=20,max_distance=1000,sparse=False,n_permutation=10)

In [ ]:
fig = tc.pl.co_occurrence(pucks, 'type-type', log_base=2, wspace=0.25);
fig.savefig(f'{data_path}/cooc_line.pdf',bbox_inches='tight')

In [ ]:
fig = tc.pl.co_occurrence_matrix(pucks, 'type-type', score_key='z', restrict_intervals=[0,1,3,7],cmap_vmin_vmax=[-5,5], value_cluster=True, group_cluster=True);
fig.savefig(f'{data_path}/cooc_matrix.pdf',bbox_inches='tight')